In [7]:
import folium
import math
import heapq as hq
from itertools import product

Poner coordenadas en (Latitud, Longitud)

In [8]:
class Point:
    def __init__(self, _name, _x, _y):
        self.n = _name
        self.x = float(_x)
        self.y = float(_y[:-2]) 
    def name(self):
        return self.n
    def lgt(self):
        return self.x
    def lat(self):
        return self.y

In [9]:
def ReadData(filename):
    G = []
    file = open(filename)
    for line in file:
        x = line.split(",")
        G.append(Point(x[1], x[2], x[3]))
    return G

Fórmula del Haversine

R = radio de la Tierra

Δlat = lat2− lat1

Δlong = long2− long1

a = sin²(Δlat/2) + cos(lat1) · cos(lat2) · sin²(Δlong/2)

c = 2 · atan2(√a, √(1−a))

d = R · c

El radio equivolumen es de 6371 km.

In [10]:
def UCS(G, s):
    t = 0
    rad = 6371
    n = len(G)
    visited = [False]*n
    weights = [math.inf]*n
    path = [-1]*n
    queue = []
    weights[s] = 0
    hq.heappush(queue, (0, s))
    last = (math.inf, None)
    minor = (math.inf, None)
    while len(queue) > 0:
        g, u = hq.heappop(queue)
        if visited[u]: continue
        visited[u] = True
        lgt = G[u].lgt()
        lat = G[u].lat()
        t+=1
        for v in range(len(G)):
            if (not u == v) and ((v == s and t == len(G) and len(queue) == 0) or (not v == s)): 
                _lgt = G[v].lgt()
                _lat = G[v].lat()
                #x = _lgt - lgt
                #y = _lat - lat
                #a = pow(math.sin(y / 2), 2) + math.cos(lat) * math.cos(_lat) * pow(math.sin(x / 2),2)
                #c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
                #h = rad * c
                h = math.sqrt(pow((_lgt-lgt),2)+pow((_lat-lat),2))
                #print(h)
                if h < minor[0]:
                    minor = (h, v)
                    
                if h < 3:
                    f = g + h
                    if f < weights[v]:
                        weights[v] = f
                        path[v] = u
                        hq.heappush(queue, (f, v))
                        last = (f, v)
                elif v == len(G) - 1:
                    #h, w = minor
                    #f = g + h
                    #weights[w] = f
                    #path[w] = u
                    #hq.heappush(queue, (f, w))
                    #last = (f, w)
                    h, w = minor
                    f = g + h
                    if f < weights[w]:
                        weights[w] = f
                        path[w] = u
                        hq.heappush(queue, (f, w))
                        last = (f, w)
    path[s] = last[1]
    weights[s] = last[0]
    return path, weights

In [16]:
def Draw(G, pos):
    path, weights = UCS(G, pos)

    C = []
    act = pos
    _bool = True
    _color = 'green'
    i=0
    while _bool:
        C.append([G[act].lat(), G[act].lgt()])    
        folium.Marker(location=(G[act].lat(), G[act].lgt()), popup=G[act].name(), icon=folium.Icon(color=_color)).add_to(m)
        i+=1
        act = path[act]
        
        if _color =='green':
            _color = 'red'
        elif _color == 'red':
            _color = 'blue'
            
        if act == pos:
            _bool = False
            
    C.append([G[act].lat(), G[act].lgt()])  
    return C 

In [15]:
G = ReadData('testdata.tst')
pos = 0

m = folium.Map(location=[41.1, -90.3], zoom_start=4)
C = Draw(G, pos)
polyLine = folium.PolyLine(locations=C, weight=2, color='green')
m.add_child(polyLine)

Porcentaje: 16.0


In [ ]:
#Coordenadas

# Coordinates are 10 points on the great circle from Boston to
# San Francisco.
# Reference: http://williams.best.vwh.net/avform.htm#Intermediate
coordinates = [
    [42.3581, -71.0636],
    [42.82995815, -74.78991444],
    [43.17929819, -78.56603306],
    [43.40320216, -82.37774519],
    [43.49975489, -86.20965845],
    [41.4338549, -108.74485069],
    [40.67471747, -112.29609954],
    [39.8093434, -115.76190821],
    [38.84352776, -119.13665678],
    [37.7833, -122.4167]]

# Create the map and add the line
m = folium.Map(location=[41.9, -97.3], zoom_start=4)
my_PolyLine=folium.PolyLine(locations=coordinates,weight=5)
m.add_child(my_PolyLine)

In [ ]:
#Marcadores

m = folium.Map()

folium.Marker(
    location=[45.3288, -121.6625],
    popup='Mt. Hood Meadows',
    icon=folium.Icon(icon='cloud')
).add_to(m)

folium.Marker(
    location=[45.3311, -121.7113],
    popup='Timberline Lodge',
    icon=folium.Icon(color='green')
).add_to(m)

folium.Marker(
    location=[45.3300, -121.6823],
    popup='Some Other Location',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

m